In [ ]:
import math
import random
from collections import Counter

class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def predict(self, X):
        return [self._predict_single(x, self.tree) for x in X]

    def _build_tree(self, X, y, depth=0):
        n_samples = len(X)
        n_features = len(X[0]) if n_samples > 0 else 0

        # Criterios de parada
        if (depth >= self.max_depth or
            n_samples < self.min_samples_split or
            len(set(y)) == 1):
            return {'class': Counter(y).most_common(1)[0][0]}

        # Selección de características y punto de división
        best_feature, best_threshold = self._best_split(X, y)

        if best_feature is None:
            return {'class': Counter(y).most_common(1)[0][0]}

        # Dividir el conjunto de datos
        left_X, left_y, right_X, right_y = [], [], [], []
        for i in range(n_samples):
            if X[i][best_feature] <= best_threshold:
                left_X.append(X[i])
                left_y.append(y[i])
            else:
                right_X.append(X[i])
                right_y.append(y[i])

        # Construir subárboles recursivamente
        left_subtree = self._build_tree(left_X, left_y, depth + 1)
        right_subtree = self._build_tree(right_X, right_y, depth + 1)

        return {
            'feature': best_feature,
            'threshold': best_threshold,
            'left': left_subtree,
            'right': right_subtree
        }

    def _best_split(self, X, y):
        best_gini = float('inf')
        best_feature = None
        best_threshold = None

        n_features = len(X[0]) if len(X) > 0 else 0

        # Probar características aleatorias (para Random Forest)
        features = random.sample(range(n_features), min(int(math.sqrt(n_features)), n_features))

        for feature in features:
            thresholds = set([x[feature] for x in X])
            for threshold in thresholds:
                left_y, right_y = [], []
                for i in range(len(X)):
                    if X[i][feature] <= threshold:
                        left_y.append(y[i])
                    else:
                        right_y.append(y[i])

                if not left_y or not right_y:
                    continue

                gini = self._gini_index(left_y, right_y)

                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def _gini_index(self, left_y, right_y):
        n_left = len(left_y)
        n_right = len(right_y)
        n_total = n_left + n_right

        gini_left = 1.0 - sum((left_y.count(c) / n_left) ** 2 for c in set(left_y))
        gini_right = 1.0 - sum((right_y.count(c) / n_right) ** 2 for c in set(right_y))

        return (n_left / n_total) * gini_left + (n_right / n_total) * gini_right

    def _predict_single(self, x, node):
        if 'class' in node:
            return node['class']

        if x[node['feature']] <= node['threshold']:
            return self._predict_single(x, node['left'])
        else:
            return self._predict_single(x, node['right'])


class RandomForest:
    def __init__(self, n_estimators=10, max_depth=5, min_samples_split=2):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        n_samples = len(X)

        for _ in range(self.n_estimators):
            # Bootstrap sampling
            indices = [random.randint(0, n_samples-1) for _ in range(n_samples)]
            X_sample = [X[i] for i in indices]
            y_sample = [y[i] for i in indices]

            tree = DecisionTree(max_depth=self.max_depth,
                               min_samples_split=self.min_samples_split)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        predictions = []
        for x in X:
            votes = [tree.predict([x])[0] for tree in self.trees]
            majority_vote = Counter(votes).most_common(1)[0][0]
            predictions.append(majority_vote)
        return predictions


# Ejemplo de uso
if __name__ == "__main__":
    # Datos de ejemplo (OR)
    X = [[0, 0], [0, 1], [1, 0], [1, 1]]
    y = [0, 1, 1, 1]

    # Crear y entrenar el Random Forest
    rf = RandomForest(n_estimators=5, max_depth=3)
    rf.fit(X, y)

    # Predecir
    test_X = [[0, 0], [0, 1], [1, 0], [1, 1]]
    predictions = rf.predict(test_X)

    print("Predicciones:", predictions)
    print("Real:", y)

Predicciones: [0, 1, 1, 1]
Real: [0, 1, 1, 1]


In [ ]:
import math

# Compatibilidad entre Python normal y MicroPython
try:
    import urandom as random
except ImportError:
    import random

# Contador simple
def contar(lista):
    conteo = {}
    for item in lista:
        if item in conteo:
            conteo[item] += 1
        else:
            conteo[item] = 1
    return conteo

def mayoritario(lista):
    conteo = contar(lista)
    max_cuenta = -1
    clase = None
    for k in conteo:
        if conteo[k] > max_cuenta:
            max_cuenta = conteo[k]
            clase = k
    return clase

class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def predict(self, X):
        return [self._predict_single(x, self.tree) for x in X]

    def _build_tree(self, X, y, depth=0):
        n_samples = len(X)
        n_features = len(X[0]) if n_samples > 0 else 0

        if (depth >= self.max_depth or n_samples < self.min_samples_split or len(set(y)) == 1):
            return {'class': mayoritario(y)}

        best_feature, best_threshold = self._best_split(X, y)

        if best_feature is None:
            return {'class': mayoritario(y)}

        left_X, left_y, right_X, right_y = [], [], [], []
        for i in range(n_samples):
            if X[i][best_feature] <= best_threshold:
                left_X.append(X[i])
                left_y.append(y[i])
            else:
                right_X.append(X[i])
                right_y.append(y[i])

        left_subtree = self._build_tree(left_X, left_y, depth + 1)
        right_subtree = self._build_tree(right_X, right_y, depth + 1)

        return {
            'feature': best_feature,
            'threshold': best_threshold,
            'left': left_subtree,
            'right': right_subtree
        }

    def _best_split(self, X, y):
        best_gini = 1
        best_feature = None
        best_threshold = None

        n_features = len(X[0]) if len(X) > 0 else 0
        m = min(int(math.sqrt(n_features)) + 1, n_features)

        features = []
        while len(features) < m:
            f = random.getrandbits(8) % n_features
            if f not in features:
                features.append(f)

        for feature in features:
            thresholds = []
            for i in range(len(X)):
                val = X[i][feature]
                if val not in thresholds:
                    thresholds.append(val)

            for threshold in thresholds:
                left_y, right_y = [], []
                for i in range(len(X)):
                    if X[i][feature] <= threshold:
                        left_y.append(y[i])
                    else:
                        right_y.append(y[i])

                if not left_y or not right_y:
                    continue

                gini = self._gini_index(left_y, right_y)
                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def _gini_index(self, left_y, right_y):
        n_left = len(left_y)
        n_right = len(right_y)
        n_total = n_left + n_right

        def gini(y_list):
            clases = set(y_list)
            score = 1.0
            for c in clases:
                p = y_list.count(c) / len(y_list)
                score -= p * p
            return score

        return (n_left / n_total) * gini(left_y) + (n_right / n_total) * gini(right_y)

    def _predict_single(self, x, node):
        if 'class' in node:
            return node['class']
        if x[node['feature']] <= node['threshold']:
            return self._predict_single(x, node['left'])
        else:
            return self._predict_single(x, node['right'])

class RandomForest:
    def __init__(self, n_estimators=5, max_depth=5, min_samples_split=2):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        n_samples = len(X)
        for _ in range(self.n_estimators):
            indices = []
            for _ in range(n_samples):
                indices.append(random.getrandbits(8) % n_samples)
            X_sample = [X[i] for i in indices]
            y_sample = [y[i] for i in indices]
            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        predictions = []
        for x in X:
            votes = []
            for tree in self.trees:
                vote = tree.predict([x])[0]
                votes.append(vote)
            predictions.append(mayoritario(votes))
        return predictions

# Prueba simple
if __name__ == '__main__':
    X = [[0, 0], [0, 1], [1, 0], [1, 1]]
    y = [0, 1, 1, 1]

    rf = RandomForest(n_estimators=5, max_depth=3)
    rf.fit(X, y)

    test_X = [[0, 0], [0, 1], [1, 0], [1, 1]]
    pred = rf.predict(test_X)

    print("Predicciones:", pred)
    print("Reales:", y)
    print("")
    print("Jannet Ortiz Aguilar")


Predicciones: [0, 1, 1, 1]
Reales: [0, 1, 1, 1]

Jannet Ortiz Aguilar


In [ ]:
import math
import random
import json
import os

# Funciones auxiliares
def contar(lista):
    conteo = {}
    for item in lista:
        if item in conteo:
            conteo[item] += 1
        else:
            conteo[item] = 1
    return conteo

def mayoritario(lista):
    conteo = contar(lista)
    max_cuenta = -1
    clase = None
    for k in conteo:
        if conteo[k] > max_cuenta:
            max_cuenta = conteo[k]
            clase = k
    return clase

# Clase DecisionTree
class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def predict(self, X):
        return [self._predict_single(x, self.tree) for x in X]

    def _build_tree(self, X, y, depth=0):
        n_samples = len(X)
        n_features = len(X[0]) if n_samples > 0 else 0

        if (depth >= self.max_depth or n_samples < self.min_samples_split or len(set(y)) == 1):
            return {'class': mayoritario(y)}

        best_feature, best_threshold = self._best_split(X, y)

        if best_feature is None:
            return {'class': mayoritario(y)}

        left_X, left_y, right_X, right_y = [], [], [], []
        for i in range(n_samples):
            if X[i][best_feature] <= best_threshold:
                left_X.append(X[i])
                left_y.append(y[i])
            else:
                right_X.append(X[i])
                right_y.append(y[i])

        left_subtree = self._build_tree(left_X, left_y, depth + 1)
        right_subtree = self._build_tree(right_X, right_y, depth + 1)

        return {
            'feature': best_feature,
            'threshold': best_threshold,
            'left': left_subtree,
            'right': right_subtree
        }

    def _best_split(self, X, y):
        best_gini = 1
        best_feature = None
        best_threshold = None

        n_features = len(X[0]) if len(X) > 0 else 0
        m = min(int(math.sqrt(n_features)) + 1, n_features)

        features = []
        while len(features) < m:
            f = random.getrandbits(8) % n_features
            if f not in features:
                features.append(f)

        for feature in features:
            thresholds = []
            for i in range(len(X)):
                val = X[i][feature]
                if val not in thresholds:
                    thresholds.append(val)

            for threshold in thresholds:
                left_y, right_y = [], []
                for i in range(len(X)):
                    if X[i][feature] <= threshold:
                        left_y.append(y[i])
                    else:
                        right_y.append(y[i])

                if not left_y or not right_y:
                    continue

                gini = self._gini_index(left_y, right_y)
                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def _gini_index(self, left_y, right_y):
        n_left = len(left_y)
        n_right = len(right_y)
        n_total = n_left + n_right

        def gini(y_list):
            clases = set(y_list)
            score = 1.0
            for c in clases:
                p = y_list.count(c) / len(y_list)
                score -= p * p
            return score

        return (n_left / n_total) * gini(left_y) + (n_right / n_total) * gini(right_y)

    def _predict_single(self, x, node):
        if 'class' in node:
            return node['class']
        if x[node['feature']] <= node['threshold']:
            return self._predict_single(x, node['left'])
        else:
            return self._predict_single(x, node['right'])

# Clase RandomForest (completamente implementada)
class RandomForest:
    def __init__(self, n_estimators=5, max_depth=5, min_samples_split=2):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        n_samples = len(X)
        for _ in range(self.n_estimators):
            indices = []
            for _ in range(n_samples):
                indices.append(random.getrandbits(8) % n_samples)
            X_sample = [X[i] for i in indices]
            y_sample = [y[i] for i in indices]
            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        predictions = []
        for x in X:
            votes = []
            for tree in self.trees:
                vote = tree.predict([x])[0]
                votes.append(vote)
            predictions.append(mayoritario(votes))
        return predictions

# Simulación de sensores
class SensorSimulator:
    @staticmethod
    def analog_read():
        return random.randint(0, 1023)

    @staticmethod
    def digital_read():
        return random.choice([0, 1])

# Gestor de Sensores
class SensorManager:
    def __init__(self):
        self.sensors = {}
        self.data = []
        self.labels = []
        self.model = RandomForest(n_estimators=5, max_depth=5)

    def add_sensor(self, name, sensor_type):
        self.sensors[name] = sensor_type

    def read_sensors(self):
        readings = {}
        for name, sensor_type in self.sensors.items():
            if sensor_type == "analog":
                readings[name] = SensorSimulator.analog_read()
            elif sensor_type == "digital":
                readings[name] = SensorSimulator.digital_read()
        return readings

    def save_to_file(self, filename="sensor_data.json"):
        data = {
            "X": self.data,
            "y": self.labels
        }
        with open(filename, "w") as f:
            json.dump(data, f)

    def load_from_file(self, filename="sensor_data.json"):
        try:
            with open(filename, "r") as f:
                data = json.load(f)
            self.data = data["X"]
            self.labels = data["y"]
            return True
        except:
            return False

    def train_model(self):
        if len(self.data) > 0 and len(self.labels) > 0:
            self.model.fit(self.data, self.labels)
            return True
        return False

    def predict(self, sensor_readings=None):
        if sensor_readings is None:
            sensor_readings = self.read_sensors()

        features = [sensor_readings[name] for name in sorted(self.sensors.keys())]
        return self.model.predict([features])[0]

    def collect_data(self, label):
        readings = self.read_sensors()
        self.data.append([readings[name] for name in sorted(self.sensors.keys())])
        self.labels.append(label)

# Ejemplo de uso
if __name__ == '__main__':
    print("Sistema de Predicción con Sensores - Demo")
    print("----------------------------------------")

    # Configurar sistema
    sistema = SensorManager()
    sistema.add_sensor("temperatura", "analog")
    sistema.add_sensor("humedad", "analog")
    sistema.add_sensor("movimiento", "digital")

    # Recolectar datos de entrenamiento
    print("\nEntrenando modelo con datos simulados...")
    for i in range(100):
        if i < 70:  # 70% datos normales
            sistema.collect_data(0)
        else:       # 30% datos de alerta
            sistema.collect_data(1)

    # Entrenar modelo
    sistema.train_model()

    # Probar el sistema
    print("\nProbando predicciones:")
    for _ in range(5):
        lectura = sistema.read_sensors()
        prediccion = sistema.predict(lectura)
        estado = "Normal" if prediccion == 0 else "ALERTA"
        print(f"Lectura: {lectura} -> Estado: {estado}")

    print("\nDemo completada exitosamente!")
    print("")
    print("Jannet Ortiz Aguilar")

Sistema de Predicción con Sensores - Demo
----------------------------------------

Entrenando modelo con datos simulados...

Probando predicciones:
Lectura: {'temperatura': 674, 'humedad': 114, 'movimiento': 0} -> Estado: Normal
Lectura: {'temperatura': 512, 'humedad': 761, 'movimiento': 0} -> Estado: Normal
Lectura: {'temperatura': 905, 'humedad': 712, 'movimiento': 0} -> Estado: Normal
Lectura: {'temperatura': 678, 'humedad': 553, 'movimiento': 0} -> Estado: Normal
Lectura: {'temperatura': 504, 'humedad': 759, 'movimiento': 1} -> Estado: Normal

Demo completada exitosamente!

Jannet Ortiz Aguilar


In [ ]:
# enfocado en lo que se desea hacer, se anexa una aplicación randomforestclassifier para toma de datos de sensores de humedad, temperatura
import math
import random
import csv  # Usamos el módulo csv estándar de Python
import time  # En lugar de utime

# Funciones básicas para el RandomForest
def contar(lista):
    """Cuenta la frecuencia de cada clase"""
    conteo = {}
    for item in lista:
        conteo[item] = conteo.get(item, 0) + 1
    return conteo

def clase_mayoritaria(lista):
    """Determina la clase más frecuente"""
    conteo = contar(lista)
    return max(conteo.items(), key=lambda x: x[1])[0]

# Clase DecisionTree simplificada
class DecisionTree:
    def __init__(self, max_depth=3, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X, y):
        """Construye el árbol de decisión"""
        self.tree = self._construir_arbol(X, y)

    def _construir_arbol(self, X, y, profundidad=0):
        """Función recursiva para construir el árbol"""
        n_muestras = len(X)

        # Criterios de parada
        if (profundidad >= self.max_depth or
            n_muestras < self.min_samples_split or
            len(set(y)) == 1):
            return {'clase': clase_mayoritaria(y)}

        # Encontrar la mejor división
        mejor_feat, mejor_val = self._mejor_division(X, y)
        if mejor_feat is None:
            return {'clase': clase_mayoritaria(y)}

        # Dividir los datos
        X_izq, y_izq, X_der, y_der = [], [], [], []
        for i in range(n_muestras):
            if X[i][mejor_feat] <= mejor_val:
                X_izq.append(X[i])
                y_izq.append(y[i])
            else:
                X_der.append(X[i])
                y_der.append(y[i])

        # Construir subárboles
        subarbol_izq = self._construir_arbol(X_izq, y_izq, profundidad + 1)
        subarbol_der = self._construir_arbol(X_der, y_der, profundidad + 1)

        return {
            'feature': mejor_feat,
            'valor': mejor_val,
            'izq': subarbol_izq,
            'der': subarbol_der
        }

    def _mejor_division(self, X, y):
        """Encuentra la mejor división para los datos"""
        mejor_gini = 1.0
        mejor_feat = None
        mejor_val = None

        n_features = len(X[0]) if X else 0

        for feat in range(n_features):
            valores = {x[feat] for x in X}
            for val in valores:
                y_izq = [y[i] for i in range(len(X)) if X[i][feat] <= val]
                y_der = [y[i] for i in range(len(X)) if X[i][feat] > val]

                if not y_izq or not y_der:
                    continue

                gini = self._calcular_gini(y_izq, y_der)
                if gini < mejor_gini:
                    mejor_gini = gini
                    mejor_feat = feat
                    mejor_val = val

        return mejor_feat, mejor_val

    def _calcular_gini(self, y_izq, y_der):
        """Calcula el índice Gini para una división"""
        n_izq = len(y_izq)
        n_der = len(y_der)
        n_total = n_izq + n_der

        def gini(y):
            conteo = contar(y)
            return 1 - sum((c/len(y))**2 for c in conteo.values())

        return (n_izq/n_total) * gini(y_izq) + (n_der/n_total) * gini(y_der)

    def predecir_instancia(self, x, arbol=None):
        """Predice la clase para una sola instancia"""
        if arbol is None:
            arbol = self.tree

        if 'clase' in arbol:
            return arbol['clase']

        if x[arbol['feature']] <= arbol['valor']:
            return self.predecir_instancia(x, arbol['izq'])
        else:
            return self.predecir_instancia(x, arbol['der'])

    def predict(self, X):
        """Predice las clases para múltiples instancias"""
        return [self.predecir_instancia(x) for x in X]

# Clase RandomForest simplificada
class RandomForest:
    def __init__(self, n_arboles=5, max_depth=3, min_samples_split=2):
        self.n_arboles = n_arboles
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.arboles = []

    def fit(self, X, y):
        """Entrena el bosque aleatorio"""
        self.arboles = []
        n_muestras = len(X)

        for _ in range(self.n_arboles):
            # Muestreo con reemplazo (bagging)
            indices = [random.randint(0, n_muestras-1) for _ in range(n_muestras)]
            X_muestra = [X[i] for i in indices]
            y_muestra = [y[i] for i in indices]

            # Crear y entrenar árbol
            arbol = DecisionTree(self.max_depth, self.min_samples_split)
            arbol.fit(X_muestra, y_muestra)
            self.arboles.append(arbol)

    def predict(self, X):
        """Predice las clases por votación mayoritaria"""
        predicciones = []
        for x in X:
            votos = [arbol.predecir_instancia(x) for arbol in self.arboles]
            predicciones.append(clase_mayoritaria(votos))
        return predicciones

# Función para exportar a CSV
def exportar_a_csv(datos, nombre_archivo='datos_sensores.csv'):
    """Exporta datos a un archivo CSV"""
    with open(nombre_archivo, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(datos)
    print(f"Datos exportados a {nombre_archivo}")

# Ejemplo de uso con sensores simulados
def main():
    print("Sistema de RandomForest para Python Estándar")
    print("------------------------------------------")

    # 1. Generar datos de ejemplo (simulando lecturas de sensores)
    print("\nGenerando datos de entrenamiento...")
    X = []
    y = []
    for _ in range(100):
        # Simular lecturas de 3 sensores (2 analógicos, 1 digital)
        temp = random.uniform(20, 30)
        humedad = random.uniform(40, 80)
        movimiento = random.randint(0, 1)

        X.append([temp, humedad, movimiento])

        # Clase basada en condiciones simuladas
        if temp > 28 and humedad > 70 and movimiento == 1:
            y.append(1)  # Condición de alerta
        else:
            y.append(0)  # Condición normal

    # 2. Exportar datos de entrenamiento a CSV
    datos_exportar = [['temp', 'humedad', 'movimiento', 'clase']]
    for i in range(len(X)):
        datos_exportar.append(X[i] + [y[i]])

    exportar_a_csv(datos_exportar)

    # 3. Entrenar el modelo
    print("\nEntrenando RandomForest...")
    rf = RandomForest(n_arboles=5, max_depth=3)
    rf.fit(X, y)

    # 4. Probar el modelo
    print("\nProbando el modelo con nuevos datos:")
    nuevos_datos = [
        [25.5, 60.2, 0],  # Normal
        [29.1, 75.3, 1],  # Alerta
        [22.0, 45.0, 0]   # Normal
    ]

    predicciones = rf.predict(nuevos_datos)

    for i, (datos, pred) in enumerate(zip(nuevos_datos, predicciones)):
        estado = "ALERTA" if pred == 1 else "Normal"
        print(f"Muestra {i+1}: {datos} -> Predicción: {estado}")

    # Generar enlace para descargar el CSV en entornos como Colab
    try:
        from google.colab import files
        files.download('datos_sensores.csv')
    except:
        pass

    print("\nEjecución completada!")

if __name__ == '__main__':
    main()

    print("")
    print("Jannet Ortiz Aguilar")

Sistema de RandomForest para Python Estándar
------------------------------------------

Generando datos de entrenamiento...
Datos exportados a datos_sensores.csv

Entrenando RandomForest...

Probando el modelo con nuevos datos:
Muestra 1: [25.5, 60.2, 0] -> Predicción: Normal
Muestra 2: [29.1, 75.3, 1] -> Predicción: ALERTA
Muestra 3: [22.0, 45.0, 0] -> Predicción: Normal


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Ejecución completada!

Jannet Ortiz Aguilar
